In [1]:
import torch
import torch.nn as nn
from torch.nn.functional import pad
import numpy as np

# Conv1d

In [2]:
# torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, 
#                 padding=0, dilation=1, groups=1, bias=True, 
#                 padding_mode='zeros', device=None, dtype=None)

Applies a 1D convolution over an input signal composed of several input planes.

- Input: $(N,C_{in},L)$ or $(C_{in},L_{in})$
- Output: $(N,C_{out},L)$ or $(C_{out},L_{out})$

where $N$ is a batch size, $C$ denotes a number of channels, $L$ is a length of signal sequence.

## 2D input data

In [3]:
number_channel = 6
sequence_length = 5
x  = torch.arange(number_channel * sequence_length, dtype=torch.float32).reshape(number_channel, sequence_length)
print(x)

tensor([[ 0.,  1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.,  9.],
        [10., 11., 12., 13., 14.],
        [15., 16., 17., 18., 19.],
        [20., 21., 22., 23., 24.],
        [25., 26., 27., 28., 29.]])


In [4]:
# kernel_size: Size of the convolving kernel, equals 2
conv1d = nn.Conv1d(in_channels=number_channel, out_channels=4, kernel_size=2)
output = conv1d.forward(x)
print(output.shape)

torch.Size([4, 4])


In [5]:
# stride: Stride of the convolution, equals 2
conv1d_with_stride = nn.Conv1d(in_channels=number_channel, out_channels=4, kernel_size=2, stride=2)
output_with_stride = conv1d_with_stride.forward(x)
print(output_with_stride.shape)

torch.Size([4, 2])


In [6]:
# padding: Padding added to both sides of the input, equals 1
conv1d_with_padding = nn.Conv1d(in_channels=number_channel, out_channels=4, kernel_size=2, padding=1)
output_with_padding = conv1d_with_padding.forward(x)
print(output_with_padding.shape)

torch.Size([4, 6])


In [7]:
# dilation: Spacing between kernel elements, equals 2
conv1d_with_dilation = nn.Conv1d(in_channels=number_channel, out_channels=4, kernel_size=2, dilation=2)
output_with_dilation = conv1d_with_dilation.forward(x)
print(output_with_dilation.shape)

torch.Size([4, 3])


In [8]:
# groups: Number of blocked connections from input channels to output channels, equals 2
conv1d_with_groups = nn.Conv1d(in_channels=number_channel, out_channels=4, kernel_size=2, groups=2)
output_with_groups = conv1d_with_groups.forward(x)
print(output_with_groups.shape)

torch.Size([4, 4])


In [9]:
# bias: If True, adds a learnable bias to the output, equals False
conv1d_without_bias = nn.Conv1d(in_channels=number_channel, out_channels=4, kernel_size=2, bias=False)
output_without_bias = conv1d_without_bias.forward(x)
print(output_without_bias.shape)

torch.Size([4, 4])


## how does `Conv1d` work with 2D input

In [10]:
conv1d.weight.data.fill_(1.0)
conv1d.bias.data.fill_(0.0)

print('weight: {}\n'.format(conv1d.weight.data.shape), conv1d.weight.data)
print('bias: {}\n'.format(conv1d.bias.data.shape), conv1d.bias.data)

weight: torch.Size([4, 6, 2])
 tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])
bias: torch.Size([4])
 tensor([0., 0., 0., 0.])


In [11]:
output = conv1d.forward(x)
print(output)

tensor([[156., 168., 180., 192.],
        [156., 168., 180., 192.],
        [156., 168., 180., 192.],
        [156., 168., 180., 192.]], grad_fn=<SqueezeBackward1>)


In [12]:
def conv_1d(x, weight, bias, stride=1, padding=0): 
    out_channels, in_channels, kernel_size = weight.shape
    X = pad(x, pad=(padding, padding, 0, 0), mode='constant', value=0)
    Y = torch.zeros((out_channels, int(np.ceil((X.shape[1] - kernel_size + 2 * padding + 1) / stride))))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[:, stride * j:stride * j + kernel_size] * weight[i]).sum() + bias[i]
    return Y

In [13]:
print('result of conv_1d: \n', conv_1d(x, weight=conv1d.weight.data, bias=conv1d.bias.data))

result of conv_1d: 
 tensor([[156., 168., 180., 192.],
        [156., 168., 180., 192.],
        [156., 168., 180., 192.],
        [156., 168., 180., 192.]])


## how does parameter `groups` work